# Kaggle Study 35일차(spooky)
코드출처 : https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle

## Approaching (Almost) Any NLP Problem on Kaggle
이 게시물에서는 카글의 자연어 처리 문제에 대해 알아보겠습니다. 예를 들어, 우리는 이 대회의 데이터를 사용할 것입니다. 우리는 먼저 매우 기본적인 첫 번째 모델을 만든 다음 다른 기능을 사용하여 개선할 것입니다. 우리는 또한 신경망들이 얼마나 깊이 사용될 수 있는지 볼 것입니다. 그리고 일반적인 조합에 대한 몇 가지 아이디어로 이 게시물을 끝냅니다.

이 캐글은 다음 아이디어를 포함하고 있습니다.
- tfidf
- count features
- logistic regression
- naive bayes
- svm
- xgboost
- grid search
- word vectors
- LSTM
- GRU
- Ensembling  
  
참고: 이 노트북은 이 데이터 세트의 리더 보드에서 높은 점수를 얻기 위한 것이 아닙니다. 하지만 제대로 따라하면 약간의 튜닝으로 아주 높은 점수를 받을 수 있습니다. ;)

그럼, 시간을 낭비하지 않고, 제가 사용할 중요한 파이썬 모듈들을 가져오는 것부터 시작하겠습니다.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
train =  pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/spooky/train.csv')
test =  pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/spooky/test.csv')
sample =  pd.read_csv('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/spooky/sample_submission.csv')

In [3]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [5]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


이 문제는 저자를 예측해야 합니다. 즉, EAP, HPL, MWS가 텍스트로 주어집니다. 간단히 말해서, 텍스트 분류는 3개의 클래스로 분류됩니다.

이 특정 문제에 대해 Kaggle은 다중 클래스 로그 손실을 평가 지표로 지정했습니다. 이 방법은 다음과 같은 방식으로 구현됩니다(다음의 링크에서 가져옵니다 : https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py)

In [6]:
def multiclass_logloss(actual,predicted,eps=1e-15):
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0],predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i,val] = 1
        actual = actual2
        
    clip = np.clip(predicted,eps,1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows*vsota

Scikit-learn의 LabelEncoder를 사용하여 텍스트 레이블을 정수로 변환합니다(0, 12).

In [7]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

더 나아가기 전에 데이터를 교육 및 검증 세트로 나누는 것이 중요합니다. 우리는 scikit-learn의 model_selection module에서 train_test_split을 사용하여 그것을 할 수 있습니다.

In [8]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y,stratify=y,random_state=42,test_size=0.1, shuffle=True)

In [9]:
print(xtrain.shape)
print(xvalid.shape)

(17621,)
(1958,)


### Building Basic Models
우리의 첫 번째 모델을 만들기 시작하겠습니다.

우리의 첫 번째 모델은 단순 TF-IDF(용어 빈도 - 역문서 빈도)이고 그 다음에는 단순 로지스틱 회귀 분석입니다.

In [10]:
# 항상 이러한 기능부터 시작하십시오. 그들은 거의 매번 일을 해요!
tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
                      token_pattern=r'\w{1,}', ngram_range=(1,3), use_idf=1, smooth_idf=1, sublinear_tf=1,
                      stop_words='english')
# TF-IDF를 교육 및 테스트 세트 모두에 맞춥니다(준지도 학습).
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [11]:
# TFIDF에서 간단한 로지스틱 회귀 분석을 사용합니다.
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.572 


C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


자, 갑니다. 우리는 0.626의 다중 클래스 로그 손실을 가진 첫 번째 모델을 가지고 있습니다.

하지만 우리는 욕심이 많고 더 좋은 점수를 원합니다. 다른 데이터로 동일한 모형을 살펴보겠습니다.

TF-IDF를 사용하는 대신 단어 수를 기능으로 사용할 수도 있습니다. 이 작업은 Scickit-learn의 Vectorizer를 사용하여 쉽게 수행할 수 있습니다.

In [12]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# 교육 세트와 테스트 세트 모두에 대한 Count Vectorizer를 장착합니다(반지도 학습)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [13]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.527 


C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


아아, 그리고 왈라! 아아아아! 첫 번째 모델을 0.1개 개선했습니다!!!

다음으로, 고대에는 꽤 유명했던 매우 단순한 모델 - Naid Bayes를 시도해 보겠습니다.

다음 두 데이터 세트에서 Naid Bayes를 사용할 때 어떤 일이 일어나는지 살펴보겠습니다.

In [14]:
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.578 


In [15]:
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.485 


우와! 오래된 것들이 여전히 효과가 있는 것 같아요!!! 목록에 있는 오래된 알고리즘 중 하나는 SVM입니다. 일부 사람들은 SVM을 "사랑"합니다. 따라서 이 데이터 세트에서 SVM을 시도해야 합니다.

SVM은 시간이 많이 걸리기 때문에 SVM을 적용하기 전에 Single Value Decomposition을 사용하여 TF-IDF의 기능 수를 줄일 것입니다.

또한 SVM을 적용하기 전에 데이터를 표준화해야 합니다.

In [16]:
# SVD 적용, 120개의 구성 요소를 선택했습니다. 120-200개의 구성 요소가 SVM 모델에 적합합니다.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# SVD에서 얻은 데이터의 크기를 조정합니다. 확장 없이 재사용할 변수 이름을 바꿉니다.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [17]:
clf = SVC(C=1.0, probability=True) 
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.733 


아차!SVM이 이 데이터에 대해 제대로 작동하지 않는 것 같습니다..!

더 나아가기 전에, Kaggle: xgboost!에 가장 인기 있는 알고리즘을 적용하겠습니다.

In [18]:
# tf-idf에 간단한 xgboost를 장착합니다.
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:06:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.781 


In [19]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

[13:06:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.772 


In [20]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

[13:09:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.767 


In [21]:
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

[13:10:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.781 


XGBoost는 운이 없는 것 같아요! 하지만 그것은 정확하지 않습니다. 아직 하이퍼 파라미터 최적화는 하지 않았습니다. 그리고 제가 게을러서 어떻게 하는지 알려드릴 테니 알아서 하세요!;) 이 문제는 다음 섹션에서 설명합니다.

### Grid Search
하이퍼 파라미터 최적화를 위한 기술입니다. 그리 효과적이지는 않지만 사용할 그리드를 알고 있으면 좋은 결과를 얻을 수 있습니다. 이 게시물에서 일반적으로 사용해야 하는 매개 변수를 지정합니다. http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/ 이 매개 변수는 제가 주로 사용하는 매개 변수입니다. 하이퍼 파라미터 최적화에는 그만큼 효과적일 수도 있고 그렇지 않을 수도 있는 다른 많은 방법이 있습니다.

이 섹션에서는 로지스틱 회귀 분석을 사용한 그리드 검색에 대해 설명합니다.

그리드 검색을 시작하기 전에 점수 매기기 기능을 만들어야 합니다. 이 작업은 scikit-learn의 make_scorer 함수를 사용하여 수행됩니다.

In [22]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

다음에는 파이프라인이 필요합니다. 여기서 시연하기 위해 SVD, 스케일링 및 로지스틱 회귀로 구성된 파이프라인을 사용합니다. 하나의 모듈보다 더 많은 모듈을 파이프라인에 배치하여 이해하는 것이 좋습니다.)

In [23]:
# SVD 객체 생성
svd = TruncatedSVD()
    
# the standard scaler  객체 생성
scl = preprocessing.StandardScaler()

# logistic regression 모델 객체 생성
lr_model = LogisticRegression()

# 파이프라인 생성
clf = pipeline.Pipeline([('svd', svd),('scl', scl),('lr', lr_model)])

In [24]:
param_grid = {'svd__n_components' : [120, 180],'lr__C': [0.1, 1.0, 10], 'lr__penalty': ['l1', 'l2']}

In [26]:
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, refit=True, cv=2)

model.fit(xtrain_tfv, ytrain) 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)

점수는 SVM의 점수와 비슷합니다. 이 기술은 아래와 같이 tune xgboost 또는 다항 순진한 베이를 미세화하는 데 사용될 수 있습니다. 여기서 tfidf 데이터를 사용합니다.

In [28]:
nb_model = MultinomialNB()

clf = pipeline.Pipeline([('nb', nb_model)])

param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, refit=True, cv=2)

model.fit(xtrain_tfv, ytrain)
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)

이것은 원래 순진한 베이스 점수보다 8% 향상된 것입니다!

NLP 문제에서는 일반적으로 단어 벡터를 살펴봅니다. 단어 벡터는 데이터에 대한 많은 통찰력을 제공합니다. 그 점에 대해 자세히 알아보죠.

### Word Vectors
너무 세세하게 설명하지 않고, 어떻게 하면 문장 벡터를 만들 수 있는지, 어떻게 하면 그 위에 기계 학습 모델을 만들 수 있는지 설명하겠습니다. 저는 GloVe vector, word2vec, fast text의 팬입니다. 이 게시물에서는 GloVe 벡터를 사용합니다. 여기에서 GloVe 벡터를 다운로드할 수 있습니다. http://www-nlp.stanford.edu/data/glove.840B.300d.zip

In [31]:
embeddings_index = {}
f = open('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/spooky/glove.840B.300d.txt',encoding='utf8')
for line in tqdm(f):
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [03:45, 9759.55it/s] 


Found 2195892 word vectors.


In [34]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [35]:
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]


 20%|███████████████                                                             | 3483/17621 [00:07<00:17, 830.22it/s]


 74%|██████████████████████████████████████████████████████▊                   | 13064/17621 [00:14<00:02, 1750.78it/s]


100%|██████████████████████████████████████████████████████████████████████████| 17621/17621 [00:16<00:00, 1057.40it/s]

100%|████████████████████████████████████████████████████████████████████████████| 1958/1958 [00:01<00:00, 1902.36it/s]


In [36]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [37]:
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:53:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:53:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.726 


In [38]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

[17:57:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:57:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.688 


간단한 파라미터 튜닝만으로 GloVe 기능의 xgboost 점수를 향상시킬 수 있습니다! 그것으로부터 훨씬 더 많은 것을 짜낼 수 있다고 믿으세요.

## Deep Learning
하지만 지금은 딥 러닝의 시대입니다! 우리는 몇 개의 신경망을 훈련하지 않고는 살 수 없습니다. 여기서는 LSTM과 GloV 기능에 대한 단순한 조밀한 네트워크를 교육할 것입니다. 먼저 고밀도 네트워크부터 살펴보겠습니다.


In [39]:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [40]:
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [41]:
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [42]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Epoch 1/5
276/276 [==============================] - 4s 5ms/step - loss: 1.0278 - val_loss: 0.7133
Epoch 2/5
276/276 [==============================] - 1s 4ms/step - loss: 0.7085 - val_loss: 0.6693
Epoch 3/5
276/276 [==============================] - 1s 4ms/step - loss: 0.6225 - val_loss: 0.6541
Epoch 4/5
276/276 [==============================] - 1s 4ms/step - loss: 0.5797 - val_loss: 0.6709
Epoch 5/5
276/276 [==============================] - 1s 3ms/step - loss: 0.5474 - val_loss: 0.6442


더 나은 결과를 얻기 위해서는 신경망의 매개 변수를 조정하고, 더 많은 계층을 추가하고, 중퇴를 늘려야 합니다. 여기서는 xgboost보다 구현 및 실행이 빠르며 더 나은 결과를 얻을 수 있다는 것을 최적화 없이 보여 줍니다.

LSTM을 사용하여 텍스트 데이터를 토큰화해야 합니다.

In [43]:
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [44]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


100%|██████████████████████████████████████████████████████████████████████████| 25943/25943 [00:04<00:00, 5497.12it/s]


In [45]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

이제 점수가 0.5점 미만임을 알 수 있습니다. 나는 그것을 최상으로 멈추지 않고 많은 에포크들을 위해 실행했지만 당신은 최고의 반복에서 멈추기 위해 조기 정지를 사용할 수 있습니다. 얼리 스톱은 어떻게 이용하나요?

음, 꽤 쉬워요 모델을 다시 컴파일합니다.

In [46]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

NotImplementedError: Cannot convert a symbolic Tensor (lstm_1/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

한가지 질문은, 왜 그렇게 많은 중퇴자를 사용하는가 하는 것입니다. 자, 낙오자가 없거나 거의 없는 모형을 적합시키면 모형을 과대 적합시키기 시작할 것입니다:)

양방향 LSTM이 더 나은 결과를 가져올 수 있는지 확인해 보겠습니다. 케라스와 함께라면 식은 죽 먹기죠.

In [47]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

NotImplementedError: Cannot convert a symbolic Tensor (bidirectional/forward_lstm_2/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [48]:
# GRU는 glove 임베딩과 2개의 밀도가 높은 층을 가지고 있습니다.
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

NotImplementedError: Cannot convert a symbolic Tensor (gru/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

좋아요! 전에 먹었던 것보다 훨씬 낫네요! 최적화 작업을 계속하면 성능이 계속 향상됩니다. 시도해 볼 만한 가치가 있습니다. 시작과 단층화입니다. 이것은 제가 지금 생략하고 있는 것입니다.

Kaggle 세계에서는 최고 점수를 얻기 위해서는 모델들의 앙상블이 있어야 합니다. 앙상블을 조금 확인해 보겠습니다!

### Ensembling
몇 달 전 저는 간단한 앙상블을 만들었지만 완전히 개발할 시간이 없었습니다. 여기서 찾을 수 있습니다. https://github.com/abhishekkrthakur/pysembler. 이 중 일부를 사용하겠습니다.

In [49]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import os
import sys
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format="[%(asctime)s] %(levelname)s %(message)s",
    datefmt="%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger(__name__)


class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification', optimize=roc_auc_score,
                 lower_is_better=False, save_path=None):
        """
        앙상블러 in function
        :param model_dict: 모델 사전, 형식은 README를 참조하십시오.
        :param num_discs: 앙상블을 위한 접힘 수입니다.
        :param task_type: 분류 또는 회귀 분석입니다.
        :param optimize(param optimize): AUC, 로그 손실 등 최적화하는 함수입니다. y_test 및 y_pred 인수 두 개가 있어야 합니다.
        :param lower_is_better: 낮은 최적화 함수의 값이 더 좋거나 더 높습니다.
        :param save_path: 생성된 예측과 함께 모델 피클을 덤프할 경로 또는 없음입니다.
        """

        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path

        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None

    def fit(self, training_data, y, lentrain):
        """
        :param training_data: 테이블 형식으로 데이터를 교육합니다.
        :paramy: 이진, 다중 클래스 또는 회귀 분석입니다.
        :반환: 예측에 사용될 모형의 사슬입니다.
        """

        self.training_data = training_data
        self.y = y

        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
            logger.info("Found %d classes", self.num_classes)
            self.lbl_enc = LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)

        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((train_prediction_shape[0],
                                                          train_prediction_shape[1] * len(self.model_dict[level])))

        for level in range(self.levels):

            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
                    logger.info("Training Level %d Fold # %d. Model # %d", level, foldnum, model_num)

                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])

                    logger.info("Predicting Level %d. Fold # %d. Model # %d", level, foldnum, model_num)

                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index,
                        (model_num * self.num_classes):(model_num * self.num_classes) +
                                                       self.num_classes] = temp_train_predictions

                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
                    logger.info("Level %d. Fold # %d. Model # %d. Validation Score = %f", level, foldnum, model_num,
                                validation_score)
                    foldnum += 1
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
                logger.info("Level %d. Model # %d. Mean Score = %f. Std Dev = %f", level, model_num,
                            avg_score, std_score)

            logger.info("Saving predictions for level # %d", level)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, "train_predictions_level_" + str(level) + ".csv"),
                                        index=False, header=None)

        return self.train_prediction_dict

    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((test_prediction_shape[0],
                                                         test_prediction_shape[1] * len(self.model_dict[level])))
        self.test_data = test_data
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level - 1]
                temp_test = self.test_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):

                logger.info("Training Fulldata Level %d. Model # %d", level, model_num)
                if level == 0:
                    model.fit(temp_train[0][model_num], self.y_enc)
                else:
                    model.fit(temp_train, self.y_enc)

                logger.info("Predicting Test Level %d. Model # %d", level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num * self.num_classes): (model_num * self.num_classes) +
                                                                                        self.num_classes] = temp_test_predictions

                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, "test_predictions_level_" + str(level) + ".csv"),
                                       index=False, header=None)

        return self.test_prediction_dict

In [50]:
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')

ens.fit(train_data_dict, ytrain, lentrain=xtrain_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=xvalid_glove.shape[0])

[19:34:32] INFO Found 3 classes
[19:34:32] INFO Training Level 0 Fold # 1. Model # 0
[19:34:34] INFO Predicting Level 0. Fold # 1. Model # 0
[19:34:34] INFO Level 0. Fold # 1. Model # 0. Validation Score = 0.626621
[19:34:34] INFO Training Level 0 Fold # 2. Model # 0


C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[19:34:36] INFO Predicting Level 0. Fold # 2. Model # 0
[19:34:36] INFO Level 0. Fold # 2. Model # 0. Validation Score = 0.616454
[19:34:36] INFO Training Level 0 Fold # 3. Model # 0


C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[19:34:38] INFO Predicting Level 0. Fold # 3. Model # 0
[19:34:38] INFO Level 0. Fold # 3. Model # 0. Validation Score = 0.619625
[19:34:38] INFO Level 0. Model # 0. Mean Score = 0.620900. Std Dev = 0.004247
[19:34:38] INFO Training Level 0 Fold # 1. Model # 1


...

[19:37:29] INFO Predicting Test Level 0. Model # 1
[19:37:29] INFO Training Fulldata Level 0. Model # 2
[19:37:29] INFO Predicting Test Level 0. Model # 2
[19:37:29] INFO Training Fulldata Level 0. Model # 3


C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[19:37:29] INFO Predicting Test Level 0. Model # 3
[19:37:29] INFO Training Fulldata Level 1. Model # 0
[19:37:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:37:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:37:39] INFO Predicting Test Level 1. Model # 0


In [51]:
multiclass_logloss(yvalid, preds[1])

0.47292458035330687

따라서, 우리는 앙상블이 점수를 크게 향상시키는 것을 볼 수 있습니다! 이것은 튜토리얼로만 되어 있기 때문에 저는 당신이 리더보드에 제출할 수 있는 CSV를 제공하지 않을 것입니다.

마음에 드셨으면 좋겠어요!

P.S.: 반응이 좋으면, 여기에 더 많은 것을 넣겠습니다! :)